In [1]:
# install Pint if necessary

try:
    import pint
except ImportError:
    !pip install pint
    !pip install matplotlib

In [2]:
from scipy.stats import lognorm, poisson
import random

In [4]:
# download modsim.py if necessary

from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)
    
download('https://raw.githubusercontent.com/AllenDowney/' +
         'ModSimPy/master/modsim.py')

In [5]:
# import functions from modsim
from modsim import *

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
# contact_matrix = [[1.2, 1.5, 5.3],
#                   [2.0, 3.6, 1.5],
#                   [0.2, 2.0 ,1.2]]

contact_matrix = [1.2, 1.5, 5.3, 2.0, 3.6, 1.5, 0.2, 2.0 ,1.2]

mu = 2.2915     # Mean of the associated normal distribution
# sigma = math.sqrt(0.284199) # Standard deviation of the associated normal distribution
sigma = 0.1332

# Generate random samples from the log-ncormal distribution
def sample_tr():
    sample = lognorm.rvs(s=sigma, scale=np.exp(mu), size=1)[0]
    return sample

def sample_tc():
    mu = random.choice(contact_matrix)
    sample = poisson.rvs(mu=mu, loc=0, size=1, random_state=None)[0]
    return sample
print(sample_tc())

In [ ]:
# tc là thời gian để một người tạo ra một contact, ở đây một người một ngày tiếp súc 2.3 người nên tc là 1/2.3
# tr là thời gian cần để hồi phục
# 2.3 là trung bình cộng hệ số lamda của tất cả các cặp nhóm tuổi trong bài báo của anh Lucky
tc = 1 / 2.3
tr = 12

beta = 1/tc
gamma = 1/tr

# s là số người có nguy cơ bị lây bệnh = dân số - số người bị bệnh
# i là số người bị bệnh ban đầu = num seed case
# r là số người hồi phục
s = 14642
i = 10
r = 0
init = State(s=s, i=i, r=r)
init /= init.sum()

# t_end là thời gian giả lập
t_end = 100
system = System(init=init, t_end=t_end, beta=beta, gamma=gamma)

In [ ]:
def update_function(t, state, system):
  s, i, r = state.s, state.i, state.r
  # infected = system.beta * s * i
  # recovered = system.gamma * i
  tr = sample_tr()
  tc = sample_tc()
  gamma = 1 / tr if tr != 0 else 0
  beta = tc
  infected = beta * s * i
  recovered = gamma * i

  s -= infected
  i += infected - recovered
  r += recovered
  return State(s=s,i=i,r=r)


def run_simulation(system, update_func):
  time_frame = TimeFrame(columns=system.init.index)
  time_frame.loc[0] = system.init
  for i in range(0, system.t_end):
    time_frame.loc[i+1] = update_func(i, time_frame.loc[i], system)
  return time_frame


def plot_results(S, I, R):
    S.plot(style='--', label='Susceptible')
    I.plot(style='-', label='Infected')
    R.plot(style=':', label='Recovered')
    decorate(xlabel='Time (days)',
             ylabel='Fraction of population')

In [ ]:
results = run_simulation(system, update_function)
plot_results(results.s, results.i, results.r)